In [1]:
!pip install pandas numpy scikit-learn matplotlib seaborn tensorflow


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [5]:
# Function to calculate EMI
def calculate_emi(principal, rate, tenure):
    rate = rate / (12 * 100)  # Convert annual rate to monthly
    tenure = tenure * 12  # Convert years to months
    emi = (principal * rate * (1 + rate) ** tenure) / ((1 + rate) ** tenure - 1)
    total_payment = emi * tenure
    total_interest = total_payment - principal
    return emi, total_interest, total_payment

# Example Calculation
principal = 500000  # ₹5 Lakhs
rate = 10  # 10% annual interest
tenure = 5  # 5 years

emi, total_interest, total_payment = calculate_emi(principal, rate, tenure)
print(f"EMI: ₹{emi:.2f}, Total Interest: ₹{total_interest:.2f}, Total Payment: ₹{total_payment:.2f}")


EMI: ₹10623.52, Total Interest: ₹137411.34, Total Payment: ₹637411.34


In [7]:
# Sample dataset: Yearly interest rates over time
data = {
    "year": [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    "interest_rate": [9.5, 9.3, 9.1, 8.9, 8.7, 8.5, 8.3, 8.1, 7.9]
}
df = pd.DataFrame(data)

# Train ML Model
X = df[['year']]
y = df['interest_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Predict interest rate for 2025
future_year = [[2025]]
predicted_rate = model.predict(future_year)[0]
print(f"Predicted Interest Rate for 2025: {predicted_rate:.2f}%")


Predicted Interest Rate for 2025: 7.50%


C:\Users\Asus\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [9]:
# Reshape data for LSTM
X_lstm = np.array(df['year']).reshape(-1, 1)
y_lstm = np.array(df['interest_rate']).reshape(-1, 1)

scaler = MinMaxScaler()
X_lstm = scaler.fit_transform(X_lstm)
y_lstm = scaler.fit_transform(y_lstm)

X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], 1, 1))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], 1, 1))

# Build LSTM Model
model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

# Compile and Train
model_lstm.compile(optimizer="adam", loss="mse")
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=4, validation_data=(X_test_lstm, y_test_lstm))

# Predict future interest rate
future_year_scaled = scaler.transform([[2025]])
future_prediction = model_lstm.predict(future_year_scaled.reshape(1, 1, 1))
predicted_rate_lstm = scaler.inverse_transform(future_prediction)[0][0]

print(f"Predicted Interest Rate (LSTM) for 2025: {predicted_rate_lstm:.2f}%")


Epoch 1/20


C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 387ms/step - loss: 0.3749 - val_loss: 0.3695
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.3203 - val_loss: 0.3496
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.3056 - val_loss: 0.3300
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.2562 - val_loss: 0.3109
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.1914 - val_loss: 0.2929
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.2769 - val_loss: 0.2755
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.1792 - val_loss: 0.2594
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.2009 - val_loss: 0.2445
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1671 - val_loss: 0.2307
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.1884 - val_loss: 0.2185
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.1431 - val_loss: 0.2083
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.1110 - val_loss: 0.2001
Epoch 13/20

In [15]:
# User Input
principal = float(input("Enter Loan Amount (₹): "))
rate = float(input("Enter Annual Interest Rate (%): "))
tenure = int(input("Enter Loan Tenure (Years): "))

# Calculate EMI
emi, total_interest, total_payment = calculate_emi(principal, rate, tenure)

print("\n📌 **Loan Summary**")
print(f"EMI: ₹{emi:.2f}")
print(f"💰 Total Interest Paid: ₹{total_interest:.2f}")
print(f"✅ Total Payment: ₹{total_payment:.2f}")

# Predict Future Interest Rate
future_interest = model.predict([[2025]])[0]
print(f"\n🔮 Predicted Interest Rate for 2025: {future_interest:.2f}%")

# Financial Advice
if emi > (principal * 0.4) / 12:
    print("⚠️ High EMI detected! Consider extending your tenure or refinancing your loan.")
else:
    print("✅ Your EMI is within a healthy limit.")

# Loan Repayment Strategies
print("\n### Loan Repayment Strategies")
print("- **Snowball Method**: Pay the smallest loan first, then move to larger debts.")
print("- **Avalanche Method**: Pay off the highest interest loans first for long-term savings.")


Enter Loan Amount (₹):  400000
Enter Annual Interest Rate (%):  11
Enter Loan Tenure (Years):  5



📌 **Loan Summary**
EMI: ₹8696.97
💰 Total Interest Paid: ₹121818.15
✅ Total Payment: ₹521818.15

🔮 Predicted Interest Rate for 2025: 7.50%
✅ Your EMI is within a healthy limit.

### Loan Repayment Strategies
- **Snowball Method**: Pay the smallest loan first, then move to larger debts.
- **Avalanche Method**: Pay off the highest interest loans first for long-term savings.


C:\Users\Asus\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
